In [1]:
DATA_PATH = '../data/raw/survey_results_public.csv'

In [2]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_rows', 10000)

In [3]:
# read dataset and print shape
data_df = pd.read_csv(DATA_PATH)
data_df.shape

(83439, 48)

In [4]:
#Display random answer
#Observations: Multiple answers need to be splitted 
#Reference to the scheme needed to understand 
data_df.sample(1).iloc[0] 

ResponseId                                                                  30037
MainBranch                                         I am a developer by profession
Employment                                                     Employed full-time
Country                                                                     Italy
US_State                                                                      NaN
UK_Country                                                                    NaN
EdLevel                           Master’s degree (M.A., M.S., M.Eng., MBA, etc.)
Age1stCode                                                          11 - 17 years
LearnCode                       Other online resources (ex: videos, blogs, etc...
YearsCode                                                                       6
YearsCodePro                                                                    2
DevType                                    Developer, mobile;Developer, front-end
OrgSize         

In [5]:
# print the general information of the data fream
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ResponseId                    83439 non-null  int64  
 1   MainBranch                    83439 non-null  object 
 2   Employment                    83323 non-null  object 
 3   Country                       83439 non-null  object 
 4   US_State                      14920 non-null  object 
 5   UK_Country                    4418 non-null   object 
 6   EdLevel                       83126 non-null  object 
 7   Age1stCode                    83243 non-null  object 
 8   LearnCode                     82963 non-null  object 
 9   YearsCode                     81641 non-null  object 
 10  YearsCodePro                  61216 non-null  object 
 11  DevType                       66484 non-null  object 
 12  OrgSize                       60726 non-null  object 
 13  C

In [6]:
# get stata for the numarical columns
data_df.describe()

,ResponseId,CompTotal,ConvertedCompYearly
count,83439.000000,4.718300e+04,4.684400e+04
mean,41720.000000,2.119407e+69,1.184262e+05
std,24086.908893,4.603702e+71,5.272944e+05
min,1.000000,0.000000e+00,1.000000e+00
25%,20860.500000,1.600000e+04,2.702500e+04
50%,41720.000000,6.700000e+04,5.621100e+04
75%,62579.500000,1.400000e+05,1.000000e+05
max,83439.000000,1.000000e+74,4.524131e+07


In [7]:
# get summarize Categorical data also
data_df[["LanguageHaveWorkedWith","Currency", "DevType"]].describe(include = 'object')

,LanguageHaveWorkedWith,Currency,DevType
count,82357,61080,66484
unique,28690,146,8626
top,Python,USD\tUnited States dollar,"Developer, full-stack"
freq,1337,14363,8601


In [8]:
# investigate the questionable obojects columns

questionable_cols = ['YearsCode', 'YearsCodePro']

for col in questionable_cols:
    print(col)
    print(data_df[col].unique())
    print('**************************')
    print()

YearsCode
[nan '7' '17' '3' '4' '6' '16' '12' '15' '10' '40' '9' '26' '14' '39' '20'
 '8' '19' '5' 'Less than 1 year' '22' '2' '1' '34' '21' '13' '25' '24'
 '30' '31' '18' '38' 'More than 50 years' '27' '41' '42' '35' '23' '28'
 '11' '37' '44' '43' '36' '33' '45' '29' '50' '46' '32' '47' '49' '48']
**************************

YearsCodePro
[nan '10' '4' '5' '6' '2' '30' '9' '18' '12' '21' '1' '16'
 'Less than 1 year' '15' '3' '35' '7' '8' '17' '14' '26' '25' '20' '50'
 '34' '11' '24' '22' '13' '31' '23' '39' '41' '27' '28' '19' '33'
 'More than 50 years' '37' '29' '32' '43' '40' '38' '45' '42' '46' '36'
 '44' '47' '48' '49']
**************************



### PREPROSSING DATA

In [9]:
# Constants
EXPORT_PATH = '../data/processed/1_preprocessed_df.pkl'

REPLACE_DICT = {
    "YearsCodePro": {'Less than 1 year': 0 , 'More than 50 years' : 51} ,
    "YearsCode": {'Less than 1 year': 0 , 'More than 50 years' : 51}
}

In [10]:
def split_answer(data_series, delimiter = ';'):
    '''
    Split multiple answers in a single string
    to a list of single strings each reprosnting a single answers 
    
    Parameters: 
    • data_series (pd.Series): String series with answers 
    • delimiter (string): Another decimal integer Defaults to ','
    
    Returns: (data_series): If column contains 
    '''
    
    #sub functions
    def is_splittable(pd_series, delimiter):
        ''' Check if results multiple should be splitted - Returns boolean '''
        return data_series.str.contains(delimiter) 
    
    def split_answer(pd_series, delimiter): 
        """ Function to split single answer  """
        return data_series.str.split(delimiter) 
    
    
    # check if multiple answers exists - if none: restore original
    splitable_values = is_splittable(data_series, delimiter)
    if not splitable_values.any():
        return data_series
    
    # else splite each value to a list
    modified_series = split_answer(data_series, delimiter)
    
    # replace NAs with empty lists
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x : [])
    
    return modified_series

In [11]:
# copy row data in new dataframe
df = data_df.copy()

### 1. Replace Values and Parse

In [12]:
for col, replacement in REPLACE_DICT.items():
    df[col] = df[col].replace(replacement).astype(np.float32)

### 2. Split Multiple Answer

In [13]:
objects_cols = df.select_dtypes(include = 'object').columns.tolist()

for col in objects_cols:
    df[col] = split_answer(df[col])

###   Visual Verify Result 

In [14]:
i = df.sample(1).index[0]
print(data_df['LanguageHaveWorkedWith'].iloc[i])
print(df['LanguageHaveWorkedWith'].iloc[i])

C;C#;C++;PowerShell
['C', 'C#', 'C++', 'PowerShell']


In [17]:
i = df.sample(1).index[0]
print(data_df['DevType'].iloc[i])
print(df['DevType'].iloc[i])

Developer, mobile;Developer, front-end;Developer, full-stack;Developer, back-end;Database administrator;Data or business analyst
['Developer, mobile', 'Developer, front-end', 'Developer, full-stack', 'Developer, back-end', 'Database administrator', 'Data or business analyst']


In [16]:
i = df.sample(1).index[0]
print(data_df['YearsCodePro'].iloc[i])
print(df['YearsCodePro'].iloc[i])

1
1.0


### Export Data

In [ ]:
df.to_pickle(ORT)